In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

#### IMDb 是一個 Dataset

In [10]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

In [46]:
def show_shapes():
    print("x_train: {}".format(x_train.shape))
    print("x_test:  {}".format(x_test.shape)) 
    print("y_train: {}".format(y_train.shape))
    print("y_test:  {}".format(y_test.shape)) 

In [51]:
def show_lens():
    print("x_train: {}".format(len(x_train)))
    print("x_test:  {}".format(len(x_test)) )
    print("y_train: {}".format(len(y_train)))
    print("y_test:  {}".format(len(y_test)) )

In [53]:
def show_type():
    print("x_train: {}".format(type(x_train)))
    print("x_test:  {}".format(type(x_test)) )
    print("y_train: {}".format(type(y_train)))
    print("y_test:  {}".format(type(y_test)) )

In [56]:
show_shapes()
show_lens()
show_type()

x_train: (25000,)
x_test:  (25000,)
y_train: (25000,)
y_test:  (25000,)
x_train: 25000
x_test:  25000
y_train: 25000
y_test:  25000
x_train: <class 'numpy.ndarray'>
x_test:  <class 'numpy.ndarray'>
y_train: <class 'numpy.ndarray'>
y_test:  <class 'numpy.ndarray'>


In [48]:
print(y_train[0])
print(y_train[1])

[1 0 0 ... 0 1 0]
1
0


In [26]:
#RNN网络容易出现反向传播过程中的梯度问题。主要原因是我们通常给RNN的参数为有限的序列。

#为了实现的简便，keras只能接受长度相同的序列输入。因此如果目前序列长度参差不齐，这时需要使用pad_sequences()。该函数是将序列转化为经过填充以后的一个新序列。

In [58]:
x_train = sequence.pad_sequences(x_train, maxlen=100)
x_test = sequence.pad_sequences(x_test, maxlen=100)
# maxlen: 整数，所有序列的最大长度。长于该长度的序列将会截短，短于该长度的序列将会填充

In [59]:
show_type()

x_train: <class 'numpy.ndarray'>
x_test:  <class 'numpy.ndarray'>
y_train: <class 'numpy.ndarray'>
y_test:  <class 'numpy.ndarray'>


In [27]:
model = Sequential()

In [28]:
model.add(Embedding(10000, 128))

In [29]:
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))

#### 1.LSTM = long short term memory，主要是为了解决长序列训练过程中的梯度消失和梯度爆炸问题。 简单来说，就是相比普通的RNN，LSTM能够在更长的序列中有更好的表现

#### 2.dropout: 在 0 和 1 之间的浮点数。 单元的丢弃比例，用于输入的线性转换。

#### 3.recurrent_dropout: 在 0 和 1 之间的浮点数。 单元的丢弃比例，            用于循环层状态的线性转换。

In [30]:
model.add(Dense(1, activation='sigmoid'))

In [31]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
_________________________________________________________________


In [60]:
model.fit(x_train, y_train, batch_size=32, epochs=10,
         validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 369s 15ms/sample - loss: 0.4552 - accuracy: 0.7869 - val_loss: 0.4116 - val_accuracy: 0.8218
Epoch 2/10
25000/25000 [==============================] - 336s 13ms/sample - loss: 0.3122 - accuracy: 0.8707 - val_loss: 0.3897 - val_accuracy: 0.8415
Epoch 3/10
25000/25000 [==============================] - 336s 13ms/sample - loss: 0.2452 - accuracy: 0.9010 - val_loss: 0.3697 - val_accuracy: 0.8436
Epoch 4/10
25000/25000 [==============================] - 336s 13ms/sample - loss: 0.1945 - accuracy: 0.9259 - val_loss: 0.4088 - val_accuracy: 0.8374
Epoch 5/10
25000/25000 [==============================] - 348s 14ms/sample - loss: 0.1577 - accuracy: 0.9426 - val_loss: 0.4220 - val_accuracy: 0.8402
Epoch 6/10
25000/25000 [==============================] - 336s 13ms/sample - loss: 0.1269 - accuracy: 0.9546 - val_loss: 0.4741 - val_accuracy: 0.8350
Epoch 7/10
25000/25000 [====================

#### y_test:  (25000,) 與 (25000,1) 的區別，以及適用時機，都能丟進fit嗎?

#### JSON(JavaScript Object Notation) 是一種輕量級的數據交換格式

In [62]:
model_json = model.to_json()
open('imdb_model_architecture.json', 'w').write(model_json)
model.save_weights('imdb_model_weights.h5')